In [1]:
import matplotlib.pyplot as plt
from keras import optimizers
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
import tensorflow as tf
import numpy as np 
import cv2
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
from jupyterthemes import jtplot
import innvestigate
import shap
from deepexplain.tensorflow import DeepExplain
from sklearn.preprocessing import normalize
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)
from PIL import *
import matplotlib.cm as cm
%matplotlib inline
K.clear_session()
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.transform import resize
from sklearn.metrics.pairwise import cosine_similarity
import glob, os
from matplotlib import image
from skimage.transform import resize
from PIL import Image, ImageChops, ImageDraw, ImageFont
import math
from matplotlib import rc
import matplotlib.pylab as plt

rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

Using TensorFlow backend.
/home/amitoj/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/amitoj/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/amitoj/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/amitoj/anaconda3/lib/python3.7/site-packages/tenso

In [2]:
x_test = np.load('annot_keras_20.npy')
#x_test = np.load('annot_keras.npy')
#x_test = np.load('test_keras.npy')

In [3]:
x_mask = np.load('masks_keras.npy')

In [4]:
base = 'OCT-annotations/attributions_20_new/'
#base = 'OCT-annotations/attributions/'
base_out = 'OCT-annotations/output_paper/'
base_img = 'OCT-annotations/images/'
base_marked = 'OCT-annotations/abdul_markings/'

In [5]:
explainers = ['shap_select', 'shap_rand', 'guided_backprop', 'deep_taylor', 'input_t_gradient', 'lrp.z', 'integrated_gradients',
              'gradient', "smoothgrad", "deconvnet", 'deeplift', 'saliency', 'occlusion', 'lrp.epsilon']
explainers.sort()
for e, exp in enumerate(explainers):
    print(e, exp)

0 deconvnet
1 deep_taylor
2 deeplift
3 gradient
4 guided_backprop
5 input_t_gradient
6 integrated_gradients
7 lrp.epsilon
8 lrp.z
9 occlusion
10 saliency
11 shap_rand
12 shap_select
13 smoothgrad


In [6]:
explainer_labels = ['DcNet', 'D. Taylor', 'DLift-Res', 'Grad', 'G. Backprop', 'I*Grad', 'IG', 'LRP.E', 'LRP.Z', 'Occ64', 'Saliency', 'SHAP-R', 'SHAP-S', 'SmoothGrad']
exp_dict = dict(zip(explainers, explainer_labels))
#exp_dict

In [7]:
dis = ['CNV', 'DME', 'DRUSEN', 'Normal']

In [8]:
weight_addrs = glob.glob('../Kermany_inception/top_weights/*')

In [9]:
model = InceptionV3(include_top=True, weights= None, classes=4)
adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=0.0, amsgrad=False)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
weight_addrs = glob.glob('../Kermany_inception/top_weights/*')
#model.load_weights(weight_addrs[7])
model.load_weights(weight_addrs[0])

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
pred_arr = []
for cl in range(3):
    pred_arr.append(model.predict(x_test[cl]))
#pred_arr
Y_pred = np.argmax(np.array(pred_arr), axis = 2)

In [11]:
Y_pred

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1],
       [2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2]])

#### Plot on images of original size

In [12]:
def get_id(file):
    #print(file)
    for i, f in enumerate(file):
        if f == "-":
            break
       # print(i)
    return(file[i+1:-5])

In [15]:
"""filepath = base_marked + dis[0] + '/'
files = []
dirs = os.listdir(filepath)
dirs.sort()
filenames = []
count = 0
for file in dirs:
    if file.endswith(".jpg"):
        filenames.append(file)
        print(count, file)
        files.append(os.path.join(filepath, file))
        count+=1"""

'filepath = base_marked + dis[0] + \'/\'\nfiles = []\ndirs = os.listdir(filepath)\ndirs.sort()\nfilenames = []\ncount = 0\nfor file in dirs:\n    if file.endswith(".jpg"):\n        filenames.append(file)\n        print(count, file)\n        files.append(os.path.join(filepath, file))\n        count+=1'

In [16]:
"""filepath = base_img + dis[0] + '/'
files = []
dirs = os.listdir(filepath)
dirs.sort()
filenames = []
count = 0
for file in dirs:
    if file.endswith(".jpeg"):
        filenames.append(file)
        print(count, file)
        files.append(os.path.join(filepath, file))
        count+=1"""

'filepath = base_img + dis[0] + \'/\'\nfiles = []\ndirs = os.listdir(filepath)\ndirs.sort()\nfilenames = []\ncount = 0\nfor file in dirs:\n    if file.endswith(".jpeg"):\n        filenames.append(file)\n        print(count, file)\n        files.append(os.path.join(filepath, file))\n        count+=1'

In [17]:
cat_mark = [1, 8, 0]
cat_img = [2, 12, 1]

In [18]:
def load_marked(cat):
    filepath = base_marked + dis[cat] + '/'
    files = []
    dirs = os.listdir(filepath)
    dirs.sort()
    filenames = []
    for file in dirs:
        if file.endswith(".jpg"):
            filenames.append(file)
            files.append(os.path.join(filepath, file))
    filename = get_id(filenames[cat_mark[cat]])
    
    img = plt.imread(files[cat_mark[cat]])
    stacked_img = np.stack((img,)*3, axis=-1)
    return stacked_img.astype('float'), filename

In [19]:
def load_img(cat):
    filepath = base_img + dis[cat] + '/'
    files = []
    dirs = os.listdir(filepath)
    dirs.sort()
    filenames = []
    for file in dirs:
        if file.endswith(".jpeg"):
            filenames.append(file)
            files.append(os.path.join(filepath, file))
    filename = get_id(filenames[cat_img[cat]])
    #print(filename)
    #return plt.imread(files[num])
    img = plt.imread(files[cat_img[cat]])
    stacked_img = np.stack((img,)*3, axis=-1)
    return stacked_img.astype('float'), filename
    #return cv2.imread(files[num])

In [20]:
def put_text(img, exp):
    w, h = img.size
    d = ImageDraw.Draw(img)
    fnt = ImageFont.truetype('arial.ttf', 60)
    d.text((40, 25), exp, font = fnt, fill=(255,0,0))
    return img.copy()

In [21]:
def crop_img(org = 0):
    im = Image.open(r"tmp.png")
    w, h = im.size
    print(w,h)
    if(org == 1 or org == 2): #return original image
        left = 0
        right = w/2
    else:
        left = w/2
        right = w
    
    
    for i in range(int(left), int(left + 0.2*w), int(0.01*w)): #search left start
        strip = im.crop((i, 0, i+10, h))
        #print(strip)
        extrema = strip.convert("L").getextrema()
        if extrema[0] == 0:
            break     
    
    for j in range(int(right), int(right - 0.2*w), -int(0.01*w)): #search right end
        strip = im.crop((j-10, 0, j, h))
        #print(strip)
        extrema = strip.convert("L").getextrema()
        if extrema[0] == 0:
            break     
            
    if(org == 1):
        im = im.crop((i, 0, j, h))
        #put_text(im, "OCT")
        #im.save("tmp_crop_org.png")
        return put_text(im, "Prediction: "+ dis[Y_pred[cat][num]])
    elif(org == 2):
        im = im.crop((i, 0, j, h))
        #put_text(im, "OCT")
        #im.save("tmp_crop_org.png")
        return put_text(im, "Marking")
    im = im.crop((i, 0, j, h))
    #put_text(im, exp)
    #im.save("tmp_crop.png")
    
    return put_text(im, exp_dict[exp])

In [22]:
import numpy as np
import warnings
try:
    import matplotlib.pyplot as pl
except ImportError:
    warnings.warn("matplotlib could not be loaded!")
    pass
import colors

def img_plot(shap_values, pixel_values,exp, org = 0, labels=None, width=40, aspect=0.2, hspace='auto', labelpad=None, show=False):
    """ Plots SHAP values for image inputs.
    Parameters
    ----------
    shap_values : [numpy.array]
        List of arrays of SHAP values. Each array has the shap (# samples x width x height x channels), and the
        length of the list is equal to the number of model outputs that are being explained.
    pixel_values : numpy.array
        Matrix of pixel values (# samples x width x height x channels) for each image. It should be the same
        shape as each array in the shap_values list of arrays.
    labels : list
        List of names for each of the model outputs that are being explained. This list should be the same length
        as the shap_values list.
    width : float
        The width of the produced matplotlib plot.
    labelpad : float
        How much padding to use around the model output labels.
    show : bool
        Whether matplotlib.pyplot.show() is called before returning. Setting this to False allows the plot
        to be customized further after it has been created.
    """

    multi_output = True
    if type(shap_values) != list:
        multi_output = False
        shap_values = [shap_values]

    # make sure labels
    if labels is not None:
        assert labels.shape[0] == shap_values[0].shape[0], "Labels must have same row count as shap_values arrays!"
        if multi_output:
            assert labels.shape[1] == len(shap_values), "Labels must have a column for each output in shap_values!"
        else:
            assert len(labels.shape) == 1, "Labels must be a vector for single output shap_values."

    label_kwargs = {} if labelpad is None else {'pad': labelpad}

    # plot our explanations
    x = pixel_values
    fig_size = np.array([17 * (len(shap_values) + 1), 3.2 * (x.shape[0] + 1)])
    if fig_size[0] > width:
        fig_size *= width / fig_size[0]
    fig, axes = pl.subplots(nrows=x.shape[0], ncols=len(shap_values) + 1, figsize=fig_size)
    fig.patch.set_facecolor('black')
    if len(axes.shape) == 1:
        axes = axes.reshape(1,axes.size)
    for row in range(x.shape[0]):
        x_curr = x[row].copy()

        # make sure
        if len(x_curr.shape) == 3 and x_curr.shape[2] == 1:
            x_curr = x_curr.reshape(x_curr.shape[:2])
        if x_curr.max() > 1:
            x_curr /= 255.

        # get a grayscale version of the image
        if len(x_curr.shape) == 3 and x_curr.shape[2] == 3:
            x_curr_gray = (0.2989 * x_curr[:,:,0] + 0.5870 * x_curr[:,:,1] + 0.1140 * x_curr[:,:,2]) # rgb to gray
        else:
            x_curr_gray = x_curr

        axes[row,0].imshow(x_curr, cmap=plt.get_cmap('gray'))
        axes[row,0].axis('off')
        if len(shap_values[0][row].shape) == 2:
            abs_vals = np.stack([np.abs(shap_values[i]) for i in range(len(shap_values))], 0).flatten()
        else:
            abs_vals = np.stack([np.abs(shap_values[i].sum(-1)) for i in range(len(shap_values))], 0).flatten()
        max_val = np.nanpercentile(abs_vals, 99.9)
        for i in range(len(shap_values)):
            if labels is not None:
                axes[row,i+1].set_title(labels[row,i], **label_kwargs)
            sv = shap_values[i][row] if len(shap_values[i][row].shape) == 2 else shap_values[i][row].sum(-1)
            axes[row,i+1].imshow(x_curr_gray, cmap=plt.get_cmap('gray'), alpha=1, extent=(-1, sv.shape[1], sv.shape[0], -1))
            im = axes[row,i+1].imshow(sv, cmap=colors.red_transparent_blue, vmin=-max_val, vmax=max_val)
            axes[row,i+1].axis('off')
    if hspace == 'auto':
        fig.tight_layout()
    else:
        fig.subplots_adjust(hspace=hspace)
    """cb = fig.colorbar(im, ax=np.ravel(axes).tolist(), label="SHAP value", orientation="horizontal", aspect=fig_size[0]/aspect)
    cb.outline.set_visible(False)"""
    print(exp)
    plt.savefig("tmp.png")
    if(org == 1):
        im_heat = crop_img(org) # get original image
    elif(org == 2):
        im_heat = crop_img(org)
    else:
        im_heat = crop_img(0) #always get heatmap
    if show:
        plt.show()
    return im_heat

In [23]:
def make_grid(img_arr, cols = 5):
    ind = 0 #index
    rows = math.ceil(len(img_arr)/cols)
    img_w, img_h = img_arr[0].size
    background = Image.new('RGBA',(img_w*cols + 10 * (cols+1), img_h*rows + 10*(rows+1)), (255, 255, 255, 255))
    bg_w, bg_h = background.size
    for r in range(rows):
        for c in range(cols):
            offset = (10 + c*img_w, 10+r*img_h)
            background.paste(img_arr[ind],offset)
            ind+=1
            if(ind == len(img_arr)):
                break
    background.save(base_out + "grid_" + str(img_id) + ".png")
    print("saved at", base_out)

In [25]:
for cat in range(3):
    img_mark, im_path = load_marked(cat)
    for num in range(cat_img[cat],cat_img[cat] + 1):
        arr_img = [] #array of original and all methods
        for i, exp in enumerate(explainers):
            filename = base + exp + '_model_' + str(0) + '.npy'
            arr = np.load(filename)
            img, img_id = load_img(cat)
            #img_mod = np.array([resize(x_test[cat][num], img.shape)]) #change resizing method
            #print(img_mod.shape)
    
            heatmap = np.array([resize(arr[cat][num], img.shape)])
    
            if i == 0:
                arr_img.append(img_plot(heatmap, np.array([img]), exp, org = 1)) #to get original
                arr_img.append(img_plot(heatmap, np.array([img_mark]), exp, org = 2))
            if i == 8: #marked instead of LRP Z
                #img_mark = np.array([resize(x_test[cat][num], img_mark.shape)])
                #print(img_mark.shape)
                #arr_img.append(img_plot(heatmap, np.array([img_mark]), exp, org = 2))
                continue
            arr_img.append(img_plot(heatmap, np.array([img]), exp))
    
        make_grid(arr_img)
        plt.close('all')

deconvnet
2448 460
deconvnet
2448 460
deconvnet
2448 460
deep_taylor
2448 460
deeplift
2448 460
gradient
2448 460
guided_backprop
2448 460
input_t_gradient
2448 460
integrated_gradients
2448 460
lrp.epsilon
2448 460
occlusion
2448 460
saliency
2448 460
shap_rand
2448 460
shap_select
2448 460
smoothgrad
2448 460
saved at OCT-annotations/output_paper/
deconvnet
2448 460
deconvnet
2448 460
deconvnet
2448 460
deep_taylor
2448 460
deeplift
2448 460
gradient
2448 460
guided_backprop
2448 460
input_t_gradient
2448 460
integrated_gradients
2448 460
lrp.epsilon
2448 460
occlusion
2448 460
saliency
2448 460
shap_rand
2448 460
shap_select
2448 460
smoothgrad
2448 460
saved at OCT-annotations/output_paper/
deconvnet
2448 460
deconvnet
2448 460
deconvnet
2448 460
deep_taylor
2448 460
deeplift
2448 460
gradient
2448 460
guided_backprop
2448 460
input_t_gradient
2448 460
integrated_gradients
2448 460
lrp.epsilon
2448 460
occlusion
2448 460
saliency
2448 460
shap_rand
2448 460
shap_select
2448 460
smo